In [1]:
from trees import *
from itertools import groupby
from camzip import camzip
from camunzip import camunzip
from filecmp import cmp
from os import stat
from json import load
import arithmeticac as arith
from math import log2
import time
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline
plotly.tools.set_credentials_file(username='karthikuwc', api_key='Ti2Ixf9lXS509cNR6Eus')

init_notebook_mode(connected=True)

### 1. General Results (Non-Adaptive Coding Methods)

In [2]:
filename = 'hamlet.txt'
Nin = stat(filename).st_size

##### 1.1 Entropy of test file

In [3]:
H = lambda pr: -sum([pr[a]*log2(pr[a]) for a in pr])
with open(filename + '.czp', 'r') as fp:
    freq = load(fp)
pf = dict([(a, freq[a]/Nin) for a in freq])
print(f'Entropy: {H(pf)} bits per symbol')

Entropy: 4.449863631694343 bits per symbol


##### 1.2 Arithmetic Coding

In [15]:
method = 'arithmetic'
start = time.time()
camzip(method, filename)
camunzip(filename + '.cz' + method[0])
finish = time.time()
print(f'Time taken: {finish-start}')



Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
reg = 8.0*Nout/Nin
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Time taken: 6.2424468994140625
Length of original file: 207039 bytes
Length of compressed file: 115163 bytes
Compression rate: 4.449905573346085 bits/byte
The two files are the same


##### 1.3 Huffman Coding

In [16]:
method = 'huffman'
start = time.time()
camzip(method, filename)
camunzip(filename + '.cz' + method[0])
finish = time.time()
print(f'Time taken: {finish-start}')


Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
huf = 8.0*Nout/Nin
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Time taken: 1.2455449104309082
Length of original file: 207039 bytes
Length of compressed file: 115752 bytes
Compression rate: 4.47266457044325 bits/byte
The two files are the same


##### 1.4 Shannon-Fano Coding

In [17]:
method = 'shannon_fano'

start = time.time()
camzip(method, filename)
camunzip(filename + '.cz' + method[0])
finish = time.time()
print(f'Time taken: {finish-start}')



Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
sha = 8.0*Nout/Nin
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Time taken: 1.3464257717132568
Length of original file: 207039 bytes
Length of compressed file: 124694 bytes
Compression rate: 4.818184013639942 bits/byte
The two files are the same


### 2. Adaptive Arithmetic Coding

##### 2.1 Simulation to explore the effect of limiting adaptive model to N samples

The following simulation is conducted for a initial bias value of 0.1. The iterations are conducted over the values of 'N' in 'model_limit'. N = 0 corresponds to the fully adaptive model, NOT 0 limit. The simulation is successful if the output doesn't print 'The files are different'.

In [8]:
method = 'carithmeticac'
model_limit = [50,100,200,400,800,1600,3200,6400,12800,25600,51200,0] #0 corressponds to full adaptive model
times = []
compression = []
for i in model_limit:
    start = time.time()
    camzip(method, filename, num=i)
    camunzip(filename + '.cz' + method[0],num=i)
    finish = time.time()
    times.append(round(finish-start,6))
    Nout = stat(filename + '.cz' + method[0]).st_size
    compression.append(8.0*Nout/Nin)
    if not cmp(filename,filename+'.cuz'):
        print('The files are different')
df = pd.DataFrame()
df['Model Limit'] = np.array(model_limit)
df['Times'] = np.array(times)
df['Compression Rate'] = np.array(compression)
df.to_csv("model_test_0_1-1.csv")
df

,Model Limit,Times,Compression Rate
0,50,7.446595,6.590913
1,100,7.336657,6.075454
2,200,6.892543,5.014630
3,400,6.914430,5.053541
4,800,6.909506,4.934143
5,1600,6.868328,4.788740
6,3200,7.097572,4.608909
7,6400,7.446874,4.510725
8,12800,7.878140,4.472974
9,25600,8.994287,4.460609


The following simulation is conducted for a initial bias value of 0.1. The iterations are conducted over the values of 'N' in 'model_limit'. N = 0 corresponds to the fully adaptive model, NOT 0 limit. The simulation is successful if the output doesn't print 'The files are different'.

In [9]:
model_limit = [0,50,100,200,400,800,1600,3200,6400,12800,25600,51200] #0 corressponds to full adaptive model
times = []
compression = []
for i in model_limit:
    start = time.time()
    camzip(method, filename,b = 0.01, num=i)
    camunzip(filename + '.cz' + method[0],b = 0.01, num=i)
    finish = time.time()
    times.append(round(finish-start,6))
    Nout = stat(filename + '.cz' + method[0]).st_size
    compression.append(8.0*Nout/Nin)
    if not cmp(filename,filename+'.cuz'):
        print('The files are different')
        
df = pd.DataFrame()
df['Model Limit'] = np.array(model_limit)
df['Times'] = np.array(times)
df['Compression Rate'] = np.array(compression)
df.to_csv("model_test_3_0_1-1.csv")
df

,Model Limit,Times,Compression Rate
0,0,22.802954,4.453229
1,50,8.257687,8.574423
2,100,7.756586,7.068736
3,200,6.792676,5.231826
4,400,6.865954,5.197398
5,800,6.827889,4.980859
6,1600,6.742874,4.793454
7,3200,6.947442,4.608291
8,6400,7.121393,4.510030
9,12800,7.378564,4.473012


##### 2.2 Plot showing results of simulations in (2.1)

In [10]:
df1 = pd.read_csv("model_test_0_1-1.csv")
df2 = pd.read_csv("model_test_3_0_1-1.csv")
trace1 = go.Scatter(
    x = df1['Model Limit'].values,
    y = df1['Compression Rate'].values,
    mode = 'markers',
    text = [('Rate: '+str(df1['Compression Rate'].values[i]),'Model Size: '+str(df1['Model Limit'].values[i]))\
            for i in range(len(df1['Compression Rate'].values))],
    name = 'Model Initial Bias = 0.1',
    marker = dict(
        size = 5,
        color = 'orange',
    )
    
)

trace2 = go.Scatter(
    x = df2['Model Limit'].values,
    y = df2['Compression Rate'].values,
    mode = 'markers',
    text = [('Rate: '+str(df2['Compression Rate'].values[i]),'Model Size: '+str(df2['Model Limit'].values[i]))\
            for i in range(len(df2['Compression Rate'].values))],
    name = 'Model Initial Bias = 0.01',
    marker = dict(
        size = 5,
        color = 'blue',
    )
)

data = [trace1,trace2]

layout = go.Layout(
    title='Relationship Between Size of Probability Model and Compression Rate ',
    xaxis=dict(
        type='log',
        title='Number of characters used to create probability model',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        type='log',
        title='Compression rate (bits/byte)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    shapes= [dict(
        type='line',
        x0=50,
        x1=51200,
        y0=df1['Compression Rate'].iloc[-1],
        y1=df1['Compression Rate'].iloc[-1],
        line=dict(
            color='orange'
        )
    ),
             dict(
        type='line',
        x0=50,
        x1=51200,
        y0=df2['Compression Rate'].iloc[0],
        y1=df2['Compression Rate'].iloc[0],
        line=dict(
            color='blue',
            dash='dashdot'
        )
    )]
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)


##### 2.3 Simulation exploring the relationship between intial bias value and compression rate.

Simulation iterates over 10 'bias' values and the output shows the relationship between bias values, compression rates, and time. Simulation is successful if 'The two files are the same' prints ten times.

In [11]:
bias = [0.01,0.03,0.05,0.07,0.09,0.11,0.13,0.15,0.17,0.19]
times = []
compression = []
for i in bias:
    start = time.time()
    camzip(method, filename, b=i)
    camunzip(filename + '.cz' + method[0], b=i)
    finish = time.time()
    times.append(round(finish-start,6))
    Nout = stat(filename + '.cz' + method[0]).st_size
    compression.append(8.0*Nout/Nin)
    if cmp(filename,filename+'.cuz'):
        print('The two files are the same')
    else:
        print('The files are different')
df = pd.DataFrame()
df['Bias'] = np.array(bias)
df['Times'] = np.array(times)
df['Compression Rate'] = np.array(compression)
df.to_csv("bias_test_2_0_1-1.csv")
df

The two files are the same
The two files are the same
The two files are the same
The two files are the same
The two files are the same
The two files are the same
The two files are the same
The two files are the same
The two files are the same
The two files are the same


,Bias,Times,Compression Rate
0,0.01,22.454095,4.453229
1,0.03,22.493888,4.452842
2,0.05,22.220630,4.452726
3,0.07,22.301350,4.452688
4,0.09,22.348054,4.452688
5,0.11,22.233853,4.452688
6,0.13,22.529835,4.452726
7,0.15,22.650354,4.452765
8,0.17,22.688937,4.452804
9,0.19,22.973099,4.452842


##### 2.4 Plot showing the results of simulation in (2.3)

In [12]:
df3 = pd.read_csv("bias_test_2_0_1-1.csv")
y = [df3['Compression Rate'].iloc[i] for i in range(df3['Compression Rate'].count()) ]
trace3 = go.Scatter(
    x = df3['Bias'].values,
    y = df3['Compression Rate'].values,
    mode = 'markers',
    name = 'Adaptive Arithmetic Coding',
    marker = dict(
        size = 5,
        color = 'orange',
    )
    
)

trace4 = go.Scatter(
    x = df3['Bias'].values,
    y = [reg]*df3['Compression Rate'].count(),
    name = 'Non-Adaptive Arithmetic Coding',
    mode = 'lines',
    marker = dict(
        size = 5,
        color = 'red',
    )
    
)

trace5 = go.Scatter(
    x = df3['Bias'].values,
    y = [huf]*df3['Compression Rate'].count(),
    name = 'Non-Adaptive Huffman Coding',
    mode = 'lines',
    marker = dict(
        size = 5,
        color = 'blue',
    )
    
)

trace6 = go.Scatter(
    x = df3['Bias'].values,
    y = [sha]*df3['Compression Rate'].count(),
    name = 'Non-Adaptive Shannon-Fano Coding',
    mode = 'lines',
    marker = dict(
        size = 5,
        color = 'green',
    )
    
)

data = [trace3,trace4]

layout = go.Layout(
    title='Relationship Between Initial Bias of Probability Model and Compression Rate ',
    xaxis=dict(
        title='Initial frequency bias used for model',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Compression rate (bits/byte)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

### 3. Contextual Arithmetic Coding

This is a test of a non-adaptive contextual arithmetic code with context length 2.

##### 3.1 Creating contextual frequency table

In [4]:
f = open('hamlet.txt', 'r')
hamlet = f.read()
f.close()

f2 = [[0 for i in range(128)] for i in range(129)]

for i in range(1,len(hamlet)):
    if i == 1: 
        f2[-1][ord(hamlet[i-1])] += 1;
    else:
        f2[ord(hamlet[i-1])][ord(hamlet[i])] += 1
        
p2 = np.array(f2,dtype='f')

for l in range(len(p2)):
    if np.sum(p2[l]) != 0:
        p2[l] = p2[l]/sum(p2[l])

In [8]:
method = 'fcondarithmetic'

start = time.time()
camzip(method, filename, pc=p2)
camunzip(filename + '.cz' + method[0], pc=p2)
finish = time.time()
print(f'Time taken: {finish-start}')


Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
sha = 8.0*Nout/Nin
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Time taken: 10.611285924911499
Length of original file: 207039 bytes
Length of compressed file: 86776 bytes
Compression rate: 3.3530301054390717 bits/byte
The two files are the same


##### 3.2 Conditional entropy of test file for context length 1

$H(X_k|X_{k-1})$

In [5]:
pj = np.array(f2,dtype='f')/len(hamlet) #Joint probability distribution

def cH(p2,pj):
    h = 0
    for i in range(len(pj)):
        for j in range(len(pj[i])):
            if pj[i][j] != 0:
                h += pj[i][j]*log2(1/p2[i][j])
    return h
print(f'Conditional Entropy: {cH(p2,pj)}')

Conditional Entropy: 3.3529715339454373


##### 3.3 Size of conditional probability distribution

As this model isn't adaptive, any practical transmission solution would require the probability distribution to be sent along with the compressed source file. Thus the effective compression rate is slightly higher than that shown above.

In [6]:
print(f'Size of conditional probabilty distribution: {len(bytes(p2))} bytes')

Size of conditional probabilty distribution: 66048 bytes


In [9]:
print(f'Effective compression rate: {8.0*(Nout+len(bytes(p2)))/Nin} bits/byte')

Effective compression rate: 5.905128985360246 bits/byte


##### 3.4 Adaptive Contextual Arithmetic Coding

The following simulation uses a context length of 1.

In [3]:
method = 'gadconarithmetic'

start = time.time()
camzip(method, filename, b=0.1)
camunzip(filename + '.cz' + method[0], b=0.1)
finish = time.time()
print(f'Time taken: {finish-start}')


Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
sha = 8.0*Nout/Nin
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Time taken: 49.27709197998047
Length of original file: 207039 bytes
Length of compressed file: 88096 bytes
Compression rate: 3.4040349885770316 bits/byte
The two files are the same


The following simulation uses a context length of 2.

In [4]:
method = 'jadconarithmetic'

start = time.time()
camzip(method, filename, b=0.1)
camunzip(filename + '.cz' + method[0], b=0.1)
finish = time.time()
print(f'Time taken: {finish-start}')


Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
sha = 8.0*Nout/Nin
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Time taken: 67.92650389671326
Length of original file: 207039 bytes
Length of compressed file: 69443 bytes
Compression rate: 2.683281893749487 bits/byte
The two files are the same


##### 3.5 Conditional entropy of test file for context length 2

$H(X_k|X_{k-1},X_{k-2})$

In [7]:
f = open('hamlet.txt', 'r')
hamlet = f.read()
f.close()

f2 = [[[0 for i in range(128)] for k in range(129)] for j in range(129)]

for i in range(len(hamlet)):
    if i == 0:
        f2[128][128][ord(hamlet[i])] +=1
    elif i == 1:
        f2[128][ord(hamlet[i-1])][ord(hamlet[i])] +=1
    else:
        f2[ord(hamlet[i-2])][ord(hamlet[i-1])][ord(hamlet[i])] += 1
        
p2 = np.array(f2,dtype='f')

for l in range(len(p2)):
    for m in range(len(p2[l])):
        if np.sum(p2[l][m]) != 0:
            p2[l][m] = p2[l][m]/sum(p2[l][m])
        
pj = np.array(f2,dtype='f')/len(hamlet) #Joint probability distribution

def cH(p2,pj):
    h = 0
    for i in range(len(pj)):
        for j in range(len(pj[i])):
            for k in range(len(pj[i][j])):
                if pj[i][j][k] != 0:
                    h += pj[i][j][k]*log2(1/p2[i][j][k])
    return h
print(f'Conditional Entropy: {cH(p2,pj)}')

Conditional Entropy: 2.357435704004353


##### 3.6 Effect of larger file size on adaptive coding

The following simulation uses a context length of 2, for a test file which is the concatenation of two copies of the original test file.

In [8]:
method = 'jadconarithmetic'
filename = 'hamlet2.txt'
start = time.time()
camzip(method, filename, b=0.1)
camunzip(filename + '.cz' + method[0], b=0.1)
finish = time.time()
print(f'Time taken: {finish-start}')


Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
sha = 8.0*Nout/Nin
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Time taken: 131.99996089935303
Length of original file: 414079 bytes
Length of compressed file: 132167 bytes
Compression rate: 2.553464435530418 bits/byte
The two files are the same
